In [13]:
import pandas as pd
from sqlalchemy import create_engine

# Extract CSV files into DataFrames

In [37]:
# Load baseball csv into a dataframe
baseball_file = "Resources/mlbsalaries.csv"
baseball_df = pd.read_csv(baseball_file)
baseball_df.head()

,salary,name,total_value,pos,years,avg_annual,team
0,"$ 3,800,000",Darryl Strawberry,"$ 3,800,000",OF,1 (1991),"$ 3,800,000",LAD
1,"$ 3,750,000",Kevin Mitchell,"$ 3,750,000",OF,1 (1991),"$ 3,750,000",SF
2,"$ 3,750,000",Will Clark,"$ 3,750,000",1B,1 (1991),"$ 3,750,000",SF
3,"$ 3,625,000",Mark Davis,"$ 3,625,000",P,1 (1991),"$ 3,625,000",KC
4,"$ 3,600,000",Eric Davis,"$ 3,600,000",OF,1 (1991),"$ 3,600,000",CIN


In [46]:
# Load basketball csv into a dataframe
basketball_file = "Resources/nbasalaries.csv"
basketball_df = pd.read_csv(basketball_file)
basketball_df.columns

Index(['RK', 'NAME', 'TEAM', 'SALARY         ', 'Team Rank'], dtype='object')

In [39]:
# Load football csv into a dataframe
football_file = "Resources/nflsalaries.csv"
football_df = pd.read_csv(football_file)
football_df.head()

,name,pos,cap_hit,cap_percent,season,team
0,Russell Wilson,QB,31000000,15.53,2020,seattle-seahawks
1,Aaron Rodgers,QB,29672000,15.06,2019,green-bay-packers
2,Kirk Cousins,QB,29000000,15.12,2019,minnesota-vikings
3,Jared Goff,QB,28842682,14.15,2020,los-angeles-rams
4,Khalil Mack,OLB,26600000,12.52,2020,chicago-bears


# Transform Baseball DataFrame

In [41]:
# Create a filtered dataframe from specific columns

baseball_cols = ["name", "salary", "years", "pos", "team"]
baseball_transformed = baseball_df[baseball_cols].copy()

# Rename columns
baseball_transformed = baseball_transformed.rename(columns = {"salary": "Salary",
                                                              "name": "Name",
                                                              "pos": "Position",
                                                              "years": "Year", 
                                                              "team": "Team"})
baseball_transformed.head()
# count = baseball_transformed["pos"].value_counts()
# count

,Name,Salary,Year,Position,Team
0,Darryl Strawberry,"$ 3,800,000",1 (1991),OF,LAD
1,Kevin Mitchell,"$ 3,750,000",1 (1991),OF,SF
2,Will Clark,"$ 3,750,000",1 (1991),1B,SF
3,Mark Davis,"$ 3,625,000",1 (1991),P,KC
4,Eric Davis,"$ 3,600,000",1 (1991),OF,CIN


# Transform Basketball DataFrame

In [45]:
# Create a filtered dataframe from specific columns

basketball_cols = ["NAME", "SALARY"]
basketball_transformed = basketball_df[basketball_cols].copy()
basketball_transformed.head()

KeyError: "['SALARY'] not in index"